## 0 Install and import dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [44]:
# cap = cv2.VideoCapture(0)
# while cap.isOpened():
#     ret,frame = cap.read()
#     cv2.imshow('frame',frame)

#     if cv2.waitKey(10) & 0xFF==ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

## 1 Make detections

In [4]:
cap = cv2.VideoCapture(0)
# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()

        # Recolor image
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False

        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable=True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

        cv2.imshow('Webcam',image)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/rhythm/.local/lib/python3.10/site-packages/cv2/qt/plugins"


## 2 Determining joints

POSE_CONNECTIONS refers to predefined set of pairs of body landmarks that are used to define connections between body landmarks

In [5]:
<img src='https://i.imgur.com/3j8BPdc.png'>

SyntaxError: invalid syntax (3354833567.py, line 1)

In [33]:
cap = cv2.VideoCapture(0)
# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()

        # Recolor image
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False

        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable=True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
        except Exception as e:
            print(e)

        # Render detections
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

        cv2.imshow('Webcam',image)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## 3 Calculate angles

In [25]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle = np.abs(radians*180/np.pi)

    if angle>180.0:
        angle = 360-angle

    return angle

In [63]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [64]:
shoulder,elbow,wrist

([0.5387870669364929, 0.7665860056877136],
 [0.6652220487594604, 1.003709316253662],
 [0.7209349870681763, 1.4196479320526123])

In [65]:
calculate_angle(shoulder,elbow,wrist)

159.56238423884975

## 4 Exercises

### 4.1 Bicep curl counter

In [91]:
cap = cv2.VideoCapture(0)

# curl counter variables
counter = 0
stage = None

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()

        # Recolor image
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False

        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable=True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angle
            angle = calculate_angle(shoulder,elbow,wrist)

            # Visualize
            cv2.putText(image,str(angle),
                        tuple(np.multiply(elbow,[640,480]).astype(int)),
                              cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
            # Count curls
            if angle > 160:
                stage = "down"
            if angle < 30 and stage=="down":
                stage="Up"
                counter+=1
                # print(counter) 
        except Exception as e:
            pass

        # Render curl counter
        # setup status boxes
        cv2.rectangle(image,(0,0),(225,73),(245,117,16),-1)

        # Rep data
        cv2.putText(image,'REPS',(15,12),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,str(counter),(10,60),
                    cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2,cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image,'STAGE',(65,12),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,stage,(90,60),
                    cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2,cv2.LINE_AA)

        # Render detections
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

        cv2.imshow('Webcam',image)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

### 4.2 Squats

In [ ]:
import mediapipe as mp
import cv2
from utils import calculate_angle, mp_drawing,mp_pose
import numpy as np

cap = cv2.VideoCapture(0)

# squat counter variables
counter = 0
stage = None

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()

        # Recolor image
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False

        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable=True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            # Calculate angle
            angle = calculate_angle(hip,knee,ankle)

            # Visualize
            cv2.putText(image,str(angle),
                        tuple(np.multiply(knee,[640,480]).astype(int)),
                              cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)

            # Count squats
            if angle > 160:
                stage = "Up"
            if angle < 90 and stage=="Up":
                stage="Down"
                counter+=1
        except Exception as e:
            print(e)

        # Render squats counter
        # setup status boxes
        cv2.rectangle(image,(0,0),(225,73),(245,117,16),-1)

        # Rep data
        cv2.putText(image,'REPS',(15,12),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,str(counter),(10,60),
                    cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2,cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image,'STAGE',(65,12),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,stage,(90,60),
                    cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2,cv2.LINE_AA)

        # Render detections
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))

        cv2.imshow('Webcam',image)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [74]:
counter

5